WGAN及其增强版
===

# 1.原始GAN的问题
## 1.1.问题一
我们知道原始GAN的损失函数为
$$-\mathbb{E}_{x\sim P_r}[\log D(x)] - \mathbb{E}_{x\sim P_g}[\log(1-D(x))]$$
判别器越好，生成器梯度消失越严重。根据原始GAN定义的判别器loss，我们可以得到最优判别器的形式；而在最优判别器的下，我们可以把原始GAN定义的生成器loss等价变换为最小化真实分布$P_r$与生成分布$P_g$之间的JS散度。我们越训练判别器，它就越接近最优，最小化生成器的loss也就会越近似于最小化$P_r$和$P_g$之间的JS散度。我们会希望如果两个分布之间越接近它们的JS散度越小，我们通过优化JS散度就能将$P_g$“拉向”$P_r$，最终以假乱真。这个希望在两个分布有所重叠的时候是成立的，但是如果两个分布完全没有重叠的部分，或者它们重叠的部分可忽略，他们的JS散度就会变为常数$log_22$。换句话说，无论$P_r$跟$P_g$是远在天边，还是近在眼前，只要它们俩没有一点重叠或者重叠部分可忽略，JS散度就固定是常数$\log_22$，而这对于梯度下降方法意味着梯度为0。此时对于最优判别器来说，生成器肯定是得不到一丁点梯度信息的；即使对于接近最优的判别器来说，生成器也有很大机会面临梯度消失的问题，最关键的是$P_r$与$P_g$不重叠或重叠部分可忽略的可能性非常大。所以我们就会得到如下结论
> 判别器训练得太好，生成器梯度消失，生成器loss降不下去；判别器训练得不好，生成器梯度不准，四处乱跑。只有判别器训练得不好不坏才行，但是这个火候又很难把握，甚至在同一轮训练的前后不同阶段这个火候都可能不一样，所以GAN才那么难训练

## 1.2.问题二
作者后来提出了一个新的loss:
$$\mathbb{E}_{x\sim P_g}[- \log D(x)]$$
同样也会导致两个问题，一句话概括：**最小化第二种生成器loss函数，会等价于最小化一个不合理的距离衡量，导致两个问题，一是梯度不稳定，二是collapse mode即多样性不足。**

### 1.2.1.梯度不稳定
上述公式经过推导可以得到下面的式子：
$$KL(P_g || P_r) - 2JS(P_r || P_g)$$
这个等价最小化目标存在两个严重的问题。第一是它同时要最小化生成分布与真实分布的KL散度，却又要最大化两者的JS散度，一个要拉近，一个却要推远！这在直观上非常荒谬，在数值上则会导致梯度不稳定，这是后面那个JS散度项的毛病。

### 1.2.2.多样性不足
前面那个正常的KL散度项也有毛病。因为KL散度不是一个对称的衡量，$KL(P_g || P_r)$与$KL(P_r || P_g)$是有差别的。以前者为例
- 当$P_g(x)\rightarrow 0$而$P_r(x)\rightarrow 1$时，$P_g(x) \log \frac{P_g(x)}{P_r(x)} \rightarrow 0$，对$KL(P_g || P_r)$贡献趋近0
- 当$P_g(x)\rightarrow 1$而$P_r(x)\rightarrow 0$时，$P_g(x) \log \frac{P_g(x)}{P_r(x)} \rightarrow +\infty$，对$KL(P_g || P_r)$贡献趋近正无穷
换言之，$KL(P_g || P_r)$对于上面两种错误的惩罚是不一样的，第一种错误对应的是“生成器没能生成真实的样本”，惩罚微小；第二种错误对应的是“生成器生成了不真实的样本” ，惩罚巨大。第一种错误对应的是缺乏多样性，第二种错误对应的是缺乏准确性。这一放一打之下，生成器宁可多生成一些重复但是很“安全”的样本，也不愿意去生成多样性的样本，因为那样一不小心就会产生第二种错误，得不偿失。这种现象就是大家常说的collapse mode-多样性不足。

## 1.3.原始GAN的问题总结
在原始GAN的(近似)最优判别器下，第一种生成器loss面临梯度消失问题，第二种生成器loss面临优化目标荒谬、梯度不稳定、对多样性与准确性惩罚不平衡导致mode collapse这几个问题。

# 2.WGAN的提出
## 2.1.Wasserstein距离
Wasserstein距离又叫Earth-Mover（EM）距离，定义如下：
$$W(P_r, P_g) = \inf_{\gamma \sim \Pi (P_r, P_g)} \mathbb{E}_{(x, y) \sim \gamma} [||x - y||]$$
解释如下：$\Pi (P_r, P_g)$是$P_r$和$P_g$组合起来的所有可能的联合分布的集合，反过来说，$\Pi (P_r, P_g)$中每一个分布的边缘分布都是$P_r$和$P_g$。对于每一个可能的联合分布$\gamma$而言，可以从中采样$(x, y) \sim \gamma$得到一个真实样本x和一个生成样本y，并算出这对样本的距离$||x-y||$，所以可以计算该联合分布$\gamma$下样本对距离的期望值$\mathbb{E}_{(x, y) \sim \gamma} [||x - y||]$。在所有可能的联合分布中能够对这个期望值取到的下界$\inf_{\gamma \sim \Pi (P_r, P_g)} \mathbb{E}_{(x, y) \sim \gamma} [||x - y||]$，就定义为Wasserstein距离。Wasserstein距离相比KL散度、JS散度的优越性在于，即便两个分布没有重叠，Wasserstein距离仍然能够反映它们的远近。

## 2.2.改进
1. 判别器最后一层去掉sigmoid
2. 生成器和判别器的loss不取log
3. 对更新后的权重强制截断到一定范围内，比如[-0.01，0.01]，以满足论文中提到的lipschitz连续性条件。
4. 论文中也推荐使用SGD， RMSprop等优化器，不要基于使用动量的优化算法，比如adam

在WGAN中，D的任务不再是尽力区分生成样本与真实样本，而是尽量拟合出样本间的Wasserstein距离，从分类任务转化成回归任务。而G的任务则变成了尽力缩短样本间的Wasserstein距离

## 2.3.WGAN的作用
1. WGAN理论上给出了GAN训练不稳定的原因，即交叉熵(JS散度)不适合衡量具有不相交部分的分布之间的距离，转而使用wassertein距离去衡量生成数据分布和真实数据分布之间的距离，理论上解决了训练不稳定的问题。
2. 解决了模式崩溃的(collapse mode)问题，生成结果多样性更丰富。
3. 对GAN的训练提供了一个指标，此指标数值越小，表示GAN训练的越差，反之越好。可以说之前训练GAN完全就和买彩票一样，训练好了算你中奖，没中奖也不要气馁，多买几注吧。

# 3.Wassertein增强型生成对抗网络(Wassertein Generative Adversarial Networks-Gradient Penalty)
WGAN-GP是WGAN之后的改进版，主要还是改进了连续性限制的条件，因为，作者也发现将权重剪切到一定范围之后，比如剪切到[-0.01,+0.01]后，发生了这样的情况，如下图左边表示<br/>
![images](../images/02_06_02_001.png)<br/>
发现大多数的权重都在-0.01 和0.01上，这就意味了网络的大部分权重只有两个可能数，对于深度神经网络来说不能充分发挥深度神经网络的拟合能力，简直是极大的浪费。并且，也发现强制剪切权重容易导致梯度消失或者梯度爆炸，梯度消失很好理解，就是权重得不到更新信息，梯度爆炸就是更新过猛了，权重每次更新都变化很大，很容易导致训练不稳定。梯度消失与梯度爆炸原因均在于剪切范围的选择，选择过小的话会导致梯度消失，如果设得稍微大了一点，每经过一层网络，梯度变大一点点，多层之后就会发生梯度爆炸 。为了解决这个问题，并且找一个合适的方式满足lipschitz连续性条件，作者提出了使用梯度惩罚(gradient penalty)的方式以满足此连续性条件，其结果如上图右边所示.由于是对每个batch中的每一个样本都做了梯度惩罚（随机数的维度是(batchsize，1)），因此判别器中不能使用batch norm,但是可以使用其他的normalization方法，比如Layer Normalization、Weight Normalization和Instance Normalization，但是训练结果中WGAN-GP生成的图片比较模糊